# 为何使用LCEL

LCEL 可以轻松地从基本组件构建复杂的链条。 

它通过提供以下功能来实现此目的： 
1. 统一的接口：每个 LCEL 对象都实现 Runnable 接口，该接口定义一组通用的调用方法（invoke、batch、stream、ainvoke 等）。 这使得 LCEL 对象链也可以自动支持这些调用。 也就是说，每个 LCEL 对象链本身就是一个 LCEL 对象。
2. 组合原语：LCEL 提供了许多原语，可以轻松组合链、并行化组件、添加后备、动态配置链内部等。


# chain的接口

# Runnable 协议

要支持链式调用，就必须实现 **Runnable** 协议。 
这是一个标准接口，可以轻松定义自定义链并以标准方式调用它们。 

标准接口包括：

- invoke：在输入上调用链
- batch：批量执行invoke
- stream：通过生成器实现流返回

这些也有相应的异步方法：

- ainvoke
- abatch
- astream
- astream_log：打印中间步骤
- astream_events：实现事件流（在 langchain-core 0.1.14 中引入）

不同组件的输入输出：

| 组件名称    | 输入类型   | 输出类型     |
|-----------|------------|-------------|
| Prompt模板    | 提示语和字典 | PromptValue |
| Retriever | 字符串 | 文档列表 |
| Tool      | 字符串或字典（依赖于工具的实现） | 依赖于工具的实现 |
| ChatModel | 字符串，对话消息或PrompValue | ChatMessage |
| LLM       | 字符串，对话消息或PrompValue | 字符串 |
| OutputParser | LLM 或 ChatModel 的输出 | 依赖于解析器的实现 |


# 输入输出schema

**chain**的输入输出都使用**pydantic**来检验**schema**是否合规。

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

## Runnable的schema

In [10]:
prompt.input_schema()

PromptInput(topic=None)

In [2]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [ ]:
prompt.output_schema.schema()

## Chain的schema

### Chain的input_schema是链序列中第一个对象的

In [27]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [13]:
chain.first

ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me a joke about {topic}'))])

### Chain的output_schema是链序列中最后一个对象的

In [ ]:
chain.output_schema.schema()

In [12]:
chain.last

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11475eb90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x114798f40>, openai_api_key='sk-SctYLVDdKoqPLXoKlLEMjLqHG2BhuOtQgjmMWvrTDqlKJeqz', openai_proxy='')

# 接口调用

## invoke

In [ ]:
chain.invoke({"topic": "bears"})

## ainvoke

## batch

## abatch

## stream

In [37]:
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Sure! Here's a bear joke for you:

Why don't bears use cellphones?

Because they already have paws for "bear-y" good reception!

## astream

## astream_event

## astream_log